<h1>DATA LOADING</h1>

In [3]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel('apartment_export_20250502_041108.xlsx')

In [6]:
districts = [
    'Чиланзарский район', 'Юнусабадский район', 'Янгихаётский район',
    'Яккасарайский район', 'Шайхантахурский район',
    'Мирабадский район', 'Учтепинский район', 'Яшнабадский район',
    'Бектемирский район', 'Сергелийский район',
    'Мирзо-Улугбекский район', 'Алмазарский район', 'Новый Ташкентский район'
]

# Filter the DataFrame where district_name matches any of the districts
tashkent_df = df[df['district_name'].isin(districts)]

In [7]:
# Replace 'Unknown' with NaN in specific columns using .loc[]
columns_to_replace_unknown = ['foundation_name', 'layout_name', 'repair_name', 'wc_name']
tashkent_df.loc[:, columns_to_replace_unknown] = tashkent_df[columns_to_replace_unknown].replace('Unknown', np.nan)

# Replace values <= 0 with NaN in specific columns using .loc[]
columns_to_check = ['total_area', 'number_of_rooms', 'floors', 'total_floors', 
                    'price_per_sqm', 'price', 'usd_uzs_rate']
tashkent_df.loc[:, columns_to_check] = tashkent_df[columns_to_check].apply(lambda x: np.where(x <= 0, np.nan, x))
tashkent_df = tashkent_df.drop(['city_name', 'region_name'], axis=1)

/var/folders/p1/5vh1vyrn3c91hc6lp5yz185m0000gn/T/ipykernel_7180/1874548450.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[43. 63. nan ... 58. 70. 47.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tashkent_df.loc[:, columns_to_check] = tashkent_df[columns_to_check].apply(lambda x: np.where(x <= 0, np.nan, x))
/var/folders/p1/5vh1vyrn3c91hc6lp5yz185m0000gn/T/ipykernel_7180/1874548450.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1. 3. 3. ... 2. 3. 2.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tashkent_df.loc[:, columns_to_check] = tashkent_df[columns_to_check].apply(lambda x: np.where(x <= 0, np.nan, x))
/var/folders/p1/5vh1vyrn3c91hc6lp5yz185m0000gn/T/ipykernel_7180/1874548450.py:8: FutureWarning: Setting an item of incompatible dtype is dep

In [8]:
tashkent_df = tashkent_df.dropna(subset=['price', 'floors', 'number_of_rooms', 'total_area'])

In [9]:
columns_to_fill = ['foundation_name', 'layout_name', 'repair_name', 'wc_name']

def fill_missing_with_mode(df, columns, group_by='district_name'):
    for column in columns:
        mode_values = df.groupby(group_by)[column].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
        
        global_mode = df[column].mode()[0] if not df[column].mode().empty else None
        
        df[column] = df.apply(
            lambda row: mode_values[row[group_by]] if pd.isna(row[column]) and mode_values[row[group_by]] is not None 
                       else (global_mode if pd.isna(row[column]) else row[column]),
            axis=1
        )
    return df

tashkent_df = fill_missing_with_mode(tashkent_df, columns_to_fill)

In [10]:
df_preprocessing_stage = tashkent_df.copy()
df_preprocessing_stage['price_usd'] = df_preprocessing_stage['price'] / df_preprocessing_stage['usd_uzs_rate']

In [11]:
repair_name_mapping = {
    'evro': 'Евроремонт',
    'sredniy': 'Средний',
    'chernovaya': 'Черновая отделка',
    'custom':'Авторский проект',
    'kapital':'Капитал'
}

df_preprocessing_stage['repair_name'] = df_preprocessing_stage['repair_name'].replace(repair_name_mapping)
df_preprocessing_stage['year_month'] = df_preprocessing_stage['datetime'].dt.to_period('M')
df_preprocessing_stage = df_preprocessing_stage.drop(['apartment_id', 'usd_uzs_rate', 'price', 'datetime', 'price_per_sqm'], axis=1)

In [12]:
df_preprocessing_stage = df_preprocessing_stage[df_preprocessing_stage['floors'] <= df_preprocessing_stage['total_floors']]

In [13]:
df_preprocessing_stage.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221433 entries, 5 to 283370
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype    
---  ------           --------------   -----    
 0   district_name    221433 non-null  object   
 1   type_of_market   221433 non-null  object   
 2   foundation_name  221433 non-null  object   
 3   layout_name      221433 non-null  object   
 4   repair_name      221433 non-null  object   
 5   wc_name          221433 non-null  object   
 6   total_area       221433 non-null  float64  
 7   number_of_rooms  221433 non-null  float64  
 8   floors           221433 non-null  float64  
 9   total_floors     221433 non-null  float64  
 10  price_usd        221433 non-null  float64  
 11  year_month       221433 non-null  period[M]
dtypes: float64(5), object(6), period[M](1)
memory usage: 22.0+ MB


<h2>HISTOGRAMS</h2>

In [ ]:
import matplotlib.pyplot as plt

# Define the columns to plot
columns_to_check = ['total_area', 'number_of_rooms', 'floors', 'total_floors', 'price_usd']

# Set up the figure and axes
fig, axes = plt.subplots(nrows=1, ncols=len(columns_to_check), figsize=(20, 10))

# Plot each column
for i, column in enumerate(columns_to_check):
    axes[i].hist(df_preprocessing_stage[column], bins=20, color='blue', alpha=0.7)
    axes[i].set_title(column)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


<h2>CORRELATION MATRIX</h2>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Specify the columns to check
columns_to_check = ['total_area', 'number_of_rooms', 'floors', 'total_floors', 'price_usd']

# Compute the correlation matrix
correlation_matrix = df_preprocessing_stage[columns_to_check].corr()

# Print the correlation matrix
correlation_matrix

<h1>OUTLIER ELEMINATION PIPELINE</h1>

<h2>FILTRATION DEPARTING FROM BUSINESS CASE PERSPECTIVE</h2>

In [14]:
df_preprocessing_stage = df_preprocessing_stage[
    (df_preprocessing_stage['number_of_rooms'] <= 7) &
    (df_preprocessing_stage['total_floors'] <= 16)
]
df_preprocessing_stage.describe()

,total_area,number_of_rooms,floors,total_floors,price_usd
count,214039.000000,214039.000000,214039.000000,214039.000000,2.140390e+05
mean,87.886226,2.485374,4.390401,7.122552,1.059089e+05
std,1066.158380,0.999654,2.702202,3.254768,2.513391e+06
min,1.000000,1.000000,1.000000,1.000000,1.388539e-02
25%,49.000000,2.000000,2.000000,4.000000,5.813833e+04
50%,63.000000,2.000000,4.000000,7.000000,7.787845e+04
75%,81.000000,3.000000,6.000000,9.000000,1.150458e+05
max,100000.000000,7.000000,16.000000,16.000000,1.161223e+09


<h2>FUNCTION DEFINED</h2>

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from itertools import product, permutations


# Outlier elimination methods
def eliminate_outliers_zscore(df, column, threshold=3):
    return df[np.abs(zscore(df[column])) < threshold]


def eliminate_outliers_iqr(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


def eliminate_outliers_isolation_forest(df, column, contamination=0.01):
    isolation_forest = IsolationForest(contamination=contamination, random_state=42)
    df['Outlier_Flag'] = isolation_forest.fit_predict(df[[column]])
    df = df[df['Outlier_Flag'] == 1]
    return df.drop(columns=['Outlier_Flag'])


def eliminate_outliers_quantile_clipping(df, column, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = df[column].quantile(lower_quantile)
    upper_bound = df[column].quantile(upper_quantile)
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


def eliminate_outliers_mad(df, column, threshold=3):
    median = df[column].median()
    mad = np.median(np.abs(df[column] - median))
    df['MAD_Score'] = np.abs(df[column] - median) / (mad + 1e-9)
    df = df[df['MAD_Score'] < threshold]
    return df.drop(columns=['MAD_Score'])


# One-hot encoding for categorical columns
def apply_one_hot_encoding(df, categorical_columns):
    df_encoded = pd.get_dummies(df, columns=categorical_columns)
    for col in df_encoded.select_dtypes(include=['bool']).columns:
        df_encoded[col] = df_encoded[col].astype(int)
    return df_encoded


# Linear regression function
def linear_regression_analysis(df, y_axis, dataset_title):
    X = df.drop(columns=[y_axis])
    y = df[y_axis]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)

    return {'RMSE': rmse, 'MAE': mae, 'MSE': mse, 'MAPE': mape, 'R2': r2}




<h2>FUNCTION EXECUTION</h2>

In [16]:
def random_forest_analysis(processed_df, target_column, pipeline_name):
    X = processed_df.drop(columns=[target_column])
    y = processed_df[target_column]
    
    metrics = random_forest(X, y)
    
    metrics['Pipeline_Sequence'] = pipeline_name
    return metrics

def evaluate_all_combinations_permutations_rf(df, target_column, categorical_columns):
    # Columns to clean
    columns_to_clean = ['total_area', 'price_usd']
    
    outlier_methods = {
        'IQR': eliminate_outliers_iqr,
        'Isolation Forest': eliminate_outliers_isolation_forest,
        'Quantile Clipping': eliminate_outliers_quantile_clipping,
        'MAD': eliminate_outliers_mad
    }
    
    column_orders = list(permutations(columns_to_clean))
    
    method_combinations = list(product(outlier_methods.items(), repeat=len(columns_to_clean)))
    
    results = []  
    count = 0

    for column_order in column_orders:
        for method_sequence in method_combinations:
            processed_df = df.copy()
            column_max_values = {} 
            
            for column, (method_name, method) in zip(column_order, method_sequence):
                try:
                    processed_df = method(processed_df, column)
                    column_max_values[column] = processed_df[column].max()  
                except Exception as e:
                    print(f"Error applying method {method_name} to column {column}: {e}")
                    column_max_values[column] = None  
            
            row_count_after_elimination = len(processed_df)
            
            processed_df = apply_one_hot_encoding(processed_df, categorical_columns)
            
            metrics = random_forest_analysis(processed_df, target_column, "Combination Pipeline")
            metrics['Pipeline_Sequence'] = " -> ".join([f"{method_name}({column})" for column, (method_name, _) in zip(column_order, method_sequence)])
            metrics['Column_Order'] = " -> ".join(column_order)
            
            for column, max_value in column_max_values.items():
                metrics[f"Max_{column}"] = max_value
            
            metrics['Row_Count_After_Outlier_Elimination'] = row_count_after_elimination
            
            results.append(metrics)
            count += 1
            print(f"SEQ: {count}/{len(column_orders) * len(method_combinations)}")
    
    return pd.DataFrame(results)


In [17]:
categorical_columns_list = ['district_name', 'foundation_name', 'layout_name','repair_name', 'wc_name', 'year_month']
result_df_6_rf = evaluate_all_combinations_permutations_rf(df=df_preprocessing_stage, target_column = 'price_usd',categorical_columns=categorical_columns_list)

NameError: name 'random_forest' is not defined

In [329]:
result_df_6_rf.to_excel('outputs_excel_rf.xlsx')

<h2>APPLY OUTLIER ELEMINATION</h2>

In [ ]:
def evaluate_selected_sequence(df, categorical_columns):
    cleaning_sequence = [
        ('total_area', 'Quantile Clipping', eliminate_outliers_quantile_clipping),
        ('price_usd', 'Quantile Clipping', eliminate_outliers_quantile_clipping)
    ]

    processed_df = df.copy()

    for column, method_name, method in cleaning_sequence:
        try:
            processed_df = method(processed_df, column)
        except Exception as e:
            print(f"Error applying method {method_name} to column {column}: {e}")
            continue  # Skip this column if there's an error

    processed_df = apply_one_hot_encoding(processed_df, categorical_columns)

    return processed_df

categorical_columns_list = ['district_name', 'foundation_name', 'layout_name','repair_name', 'wc_name', 'year_month']
cleaned_df = evaluate_selected_sequence(df_preprocessing_stage, categorical_columns_list)

In [ ]:
cleaned_df.describe()

<h3>GRID SEARCH</h3>

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import BayesianRidge
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import VotingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


def preprocess_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def calculate_metrics(y_test, y_pred):
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return {
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "MAPE": mape,
        "R2": r2
    }


# Modified function with loss visualization
def optimized_model_with_kfold_and_loss(model, X, y, param_grid=None, model_name="Model"):
    X_train, X_test, y_train, y_test = preprocess_data(X, y)

    # K-Fold Cross Validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds
    if param_grid:
        # Grid Search Optimization with K-Fold
        grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=kfold, verbose=1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
    else:
        # No Grid Search, only K-Fold
        best_model = model
        for train_idx, val_idx in kfold.split(X_train):
            best_model.fit(X_train[train_idx], y_train[train_idx])
            val_pred = best_model.predict(X_train[val_idx])

    # Fit on the entire training set with best model
    best_model.fit(X_train, y_train)
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Metrics
    return calculate_metrics(y_test, y_test_pred), best_model






<h3>Traditional ML</h3>

In [ ]:
def optimized_decision_tree(X, y):
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    model_name = "DecisionTree"
    model = DecisionTreeRegressor(random_state=23)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

def optimized_random_forest(X, y):
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    model_name = "RandomForest"
    model = RandomForestRegressor(random_state=23)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

def optimized_knn(X, y):
    param_grid = {
        'n_neighbors': [3, 5, 10],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    }
    model_name = "KNN"
    model = KNeighborsRegressor()
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

<h3>Ensemble Methods</h3>

In [ ]:
def optimized_xgboost(X, y):
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2]
    }
    model_name = "XGBoost"
    model = XGBRegressor(random_state=23)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

def optimized_catboost(X, y):
    param_grid = {
        'iterations': [100, 200],
        'depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2]
    }
    model_name = "CatBoost"
    model = CatBoostRegressor(silent=True, random_state=23)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

def optimized_lightgbm(X, y):
    from lightgbm import LGBMRegressor
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [20, 31, 50]
    }
    model_name = "LightGBM"
    model = LGBMRegressor(random_state=23)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model


<h3>ANN</h3>

In [ ]:
def optimized_mlp(X, y):
    from sklearn.neural_network import MLPRegressor
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (100, 50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'learning_rate': ['constant', 'adaptive']
    }
    model_name = "MLP"
    model = MLPRegressor(random_state=23, max_iter=500)
    metrics, best_model = optimized_model_with_kfold_and_loss(model, X, y, param_grid=param_grid, model_name=model_name)
    return metrics, best_model

def optimized_lstm(X, y, timesteps):
    from keras.models import Sequential
    from keras.layers import LSTM, Dense
    from sklearn.model_selection import GridSearchCV
    import numpy as np

    # Reshaping data for LSTM model
    X_reshaped = np.array([X[i:i+timesteps] for i in range(len(X) - timesteps)])
    y_reshaped = y[timesteps:]

    def build_model(units, learning_rate):
        model = Sequential()
        model.add(LSTM(units, input_shape=(timesteps, X.shape[1])))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', learning_rate=learning_rate)
        return model

    param_grid = {
        'units': [50, 100],
        'learning_rate': [0.01, 0.001]
    }
    model_name = "LSTM"
    # Implementing a wrapper or using GridSearchCV with Keras models
    metrics, best_model = optimized_model_with_kfold_and_loss(build_model, X_reshaped, y_reshaped, param_grid=param_grid, model_name=model_name)
    return metrics, best_model


In [ ]:
decision_tree_metrics, dt_best_model = optimized_decision_tree(X, y)
# random_forrest_metircs, rf_best_model = optimized_random_forest(X, y)
# knn_metrics, knn_best_model = optimized_knn(X, y)
# xgboost_metrics, xgboost_best_model = optimized_xgboost(X, y)
# catboost_metrics, catboost_model = optimized_catboost(X, y)


models_measurement_optimized = {}

#Traditional ML
models_measurement_optimized['Decision Tree'] = decision_tree_metrics
# models_measurement['Random Forrest'] = random_forrest_metircs
# models_measurement['KNN'] = knn_metrics
# models_measurement['XGBoost'] = xgboost_metrics
# models_measurement['CatBoost'] = catboost_metrics



In [ ]:
lstm_metrics, lstm_best_model = optimized_mlp(X, y)(X, y)